In [23]:
import pandas as pd
path = '../Podaci/9_S/2022/zones_20_year_2022.csv'
path2 = '../Podaci/Opcenito/zone_fcu.csv'
zone = pd.read_csv(path, low_memory=False)
fcu = pd.read_csv(path2)

In [30]:
import pandas as pd
zone_ids_fcu1 = fcu[fcu['fcu_type_id'] == 1.0]['zone_id'].unique().tolist()

zone['timestamp'] = pd.to_datetime(zone['timestamp'])

piv = zone.pivot_table(index='timestamp', 
                       columns='zone_id', 
                       values='zone_fan_speed', 
                       aggfunc='first')

piv = piv.sort_index()
piv = piv.bfill()

piv_fokus = piv[zone_ids_fcu1]

changes_any_zone = piv_fokus.diff().ne(0).any(axis=1)

piv['segment_id'] = changes_any_zone.cumsum()

segments = piv.groupby('segment_id', as_index=False).agg(
    start_time = ('segment_id', lambda x: x.index[0]),
    end_time   = ('segment_id', lambda x: x.index[-1])
)

segments['duration'] = segments['end_time'] - segments['start_time']
segments_15min = segments[segments['duration'] >= pd.Timedelta(minutes=15)]
